In [3]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
import numpy as np

In [4]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [5]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [6]:
r_new = scale_recurrence(r)

In [7]:
max_abs = .0000001
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
g = sp.Function("g")
n = sp.symbols("n")

In [8]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [9]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(15)

In [10]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        subs_dict[g(i)] = get_recurrence(recur.subs(rct, rct_val), i).subs(subs_dict).subs(coord_dict)
        print(get_recurrence(recur.subs(rct, 1),i))
    return np.array(list(subs_dict.values()))

In [11]:
def evaluate_recurrence_lamb(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = get_recurrence(recur.subs(rct, rct_val), i)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [12]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        exp = (derivs[i]*rct_val**i)
        f = sp.lambdify(var, exp)
        retMe.append(f(coord_dict[var[0]], coord_dict[var[1]]))
    return np.array(retMe)

In [13]:
def compute_error(pw):
    x_coord = 0.11302666666666661
    y_coord = 0
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = 1
    exp = evaluate_recurrence_lamb(coord_dict, rct_val, r_new, 14)
    true = evaluate_true(coord_dict, rct_val, 14)
    print(exp)
    print(true)
    return np.abs(exp-true)/np.abs(true)

In [14]:
compute_error(7)

[-2.18013149991004 -8.84746962368763 -78.2777187420753 -1385.11947756415
 -36764.4075087807 -1301087.91466724 -57556679.0138276 -3055385815.29108
 -189227032326.038 -13393443363881.4 -1.06648274986670e+15
 -9.43567373363252e+16 -9.18300204110757e+18 -9.74955979355538e+20]
[-2.18013150e+00 -8.84746962e+00 -7.82777187e+01 -1.38511948e+03
 -3.67644075e+04 -1.30108791e+06 -5.75566790e+07 -3.05538582e+09
 -1.89227032e+11 -1.33934434e+13 -1.06648275e+15 -9.43567373e+16
 -9.18300204e+18 -9.74955979e+20]


array([0, 2.00775692367946e-16, 0, 1.64154557874739e-16,
       7.91630613108144e-16, 2.14740886634168e-15, 1.03558172216766e-15,
       2.18490253552735e-15, 4.03187347889313e-15, 6.99969361522217e-15,
       3.30056909072257e-13, 2.10944131408974e-13, 2.03840965255176e-13,
       1.33202052553871e-12], dtype=object)

In [68]:
pw = 8
x0 = 1
x1 = 10**(-pw)
g_1 = -5000000000.00000
g_2 = (x0**2 - x1**2)*g_1/(x0**3 + x0*x1**2)
g_2

-0.0

In [72]:
derivs[7]

-720*x0*(64*x0**6/(x0**2 + x1**2)**3 - 112*x0**4/(x0**2 + x1**2)**2 + 56*x0**2/(x0**2 + x1**2) - 7)/(x0**2 + x1**2)**4

In [71]:
derivs[2].subs(var[0], 10**(-8)).subs(var[1], 10**(-8))

0.555111512312578

In [51]:
g_2

-0.0

In [14]:
'''
x_plot = [i for i in range(len(compute_error(0)))]
for i in range(1, 4):
    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))
plt.xlabel("order of derivative being computed")
plt.ylabel("absolute error")
plt.title("recurrence error vs order for different source-locations")
plt.legend(title='ratio of x_{coord_src}/y_{coord_src}')
plt.show()
'''

'\nx_plot = [i for i in range(len(compute_error(0)))]\nfor i in range(1, 4):\n    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))\nplt.xlabel("order of derivative being computed")\nplt.ylabel("absolute error")\nplt.title("recurrence error vs order for different source-locations")\nplt.legend(title=\'ratio of x_{coord_src}/y_{coord_src}\')\nplt.show()\n'